In [ ]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time

get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
import pickle
import math
import glob
import talib as ta

#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [((datetime.datetime.fromisoformat(res['time'][:19]))+datetime.timedelta(hours=+9)).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

def logarism(x):
    return math.log(x)


def get_now_data(time, period_Days, money='USD_JPY', foot = 'D', back_days = 0):
    
    all_data = []
    #time = time.replace(hour=7, minute=0, second=0, microsecond=0)
    #入力された日の当日の7時
    
    global PAST
    past = datetime.timedelta(hours=-PAST)
    
    NY = datetime.timedelta(hours=-14)
    back = datetime.timedelta(days=-back_days)
        
    date_to = time + NY + datetime.timedelta(seconds=-1) + back + past
#     date_from = date_to + datetime.timedelta(days=-period_Days)
    date_from = date_to + datetime.timedelta(hours=-period_Days)
    
    ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)

    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    
    
    close = np.array(df["Close"]).astype(float)
    #以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる
    #単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
    df["SMA5"]= ta.SMA(close, timeperiod=5) / close * 100
    df["SMA10"]= ta.SMA(close, timeperiod=10) / close * 100
    #df["RSI"] = ta.RSI(close, timeperiod=9) / close * 1000
    #ボリンジャーバンド 
    upper, middle, lower = ta.BBANDS(close, timeperiod=10, nbdevup=2, nbdevdn=2)
    df["BBANDS+σ"] = upper / close * 100
    df["BBANDS-σ"] = lower / close * 100
    
    #RSI, RATE 倍率変更 12/7
    
#     df['Rate']=(df['Close']-df['Open'])/df['Open']
    
#     df['Open'] = df['Open'].apply(lambda x: logarism(x))
#     df['Close'] = df['Close'].apply(lambda x: logarism(x))
    
    df['Rate']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    df['Rate']=df['Rate']*100
    
    df['Rate']=df['Rate']*1000
    
    df = df.dropna()
    
    df = df.set_index('Datetime')
    #print(df)
    
    
    return df
    
def get_csv(back_days, hours, money):
    
    time = datetime.datetime.now() 
    
    df = get_now_data(time, period_Days=hours, money=money, foot = FOOT , back_days=back_days)
    date_to = (time+datetime.timedelta(days=-back_days)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(hours=-hours)+datetime.timedelta(days=-back_days)).strftime("%Y-%m-%d")
    df.to_csv(str(money)+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv', mode = 'w')
    print('"'+str(money)+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv', end='", ')
    
    return df

def get_moneys_csv(back_days, hours, moneylist):
    csvlist = []
    
    time = datetime.datetime.now()
    date_to = (time+datetime.timedelta(days=-back_days)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(hours=-hours)+datetime.timedelta(days=-back_days)).strftime("%Y-%m-%d")
    
    for i in range (len(moneylist)):
        get_csv(back_days, hours, money=moneylist[i])
        csvlist.extend([str(moneylist[i])+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv'])
    
    return csvlist
        
def classify_two(x, c):
        if x<=0:
            c[0]+=1
            return -1
        elif 0<x:
            c[1]+=1
            return 1
        

def predict(dataname, moneyname, test_days, back_day, datanum):

    df = pd.read_csv(dataname, index_col='Datetime')
    
    count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df['class'] = (df["Rate"]).apply(lambda x: classify_two(x, count))
    df['class'] = df['class'].shift(-1)
    print("Train Data")
    print(" LOW/HIGH")
    print(count)
    
    X = df.drop('class', axis=1)
    y = df['class']
    X = X.drop(X.index[len(X)-1])
    y = y.dropna()
    
#     df_now = get_now_data(datetime.datetime.now(), period_Days=test_days, money=moneyname, foot = 'D', back_days = back_day)
    df_now = get_now_data(datetime.datetime.now(), period_Days=test_days, money=moneyname, foot = FOOT, back_days = back_day)
    print("money: "+moneyname)
    
    count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(test_days)+ "days ago")
    print(" LOW/MID/HIGH")
    print(count)
    
    print("learning data")
    print(df.head(1).index.values, end=' to ')
    print(df.tail(1).index.values)
    
    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    X_train = X
    y_train = y
    X_test = X_now
    y_test = y_now
    
    
    names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"]
    classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=10),
    RandomForestClassifier(max_depth=10, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]
    
    accuracy_scores = []
    precision_scores = []
    next_class = []
    profit = []
    
    clearlist = []
    
    for name, clf in zip(names, classifiers):
        print(name)
        clf.fit(X_train, y_train)
        result = clf.predict(X_test)
        profit, values = calc_profit(result, y_test)
        precision = precision_score(y_test, result, average=None)
        accuracy = accuracy_score(y_test, result)
        confusion = confusion_matrix(y_test, result)
        
        if(precision[0]>0.45 and precision[1]>0.45 and precision[0]!=1 and precision[1]!=1 and accuracy>0.53 and abs(confusion[0][0]-confusion[1][1])<(confusion[0][0]+confusion[1][1])/4):
        #if(precision[0]>0.4 and precision[1]>0.4):
#             score = clf.score(X_test, y_test)
            next_class.extend(clf.predict(X_next))

            print('accuracy_score: ', end='')
            print(accuracy_score(y_test, result))
            accuracy_scores.extend([accuracy_score(y_test, result)])
            print('confusion_matrix: ')
            print(confusion_matrix(y_test, result))
            print('precision_score: ', end='')
            print(precision_score(y_test, result, average=None))
            precision_scores.extend([precision_score(y_test, result, average=None)])

            print(str(len(result))+"days trade")
            print("profit: "+str(profit))

            print('')

            model_name = moneyname + "_" + name + "_" + str(datanum) +".sav"
            #clearlist.extend([model_name])
            print("CLEAR: " + model_name)
            pickle.dump(clf, open(model_name, 'wb'))

    #print(clearlist)
    return accuracy_scores, precision_scores, next_class, profit

def make_models(test_days, namelist, moneylist, back_days, datanum):
    for i in range(len(namelist)):
        print(namelist[i])
        acc, pre, next_class, profit = predict(namelist[i], moneylist[i], test_days, back_days, datanum)
        print(acc)


def calc_profit(predict_result, true_data):
    win=0
    lose=0
    for i in range(len(predict_result)):
            if (predict_result[i]==-1):#Low予想のとき
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
            elif(predict_result[i]==1):#High予想の時
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
    profit = win*0.95-lose #利率を算出
    values = predict_result*profit #予測結果×利率により予測の価値を算出
    return profit, values

def plot_result(index, predict_result, true_data):
    plt.figure(figsize=(20,5))
    index = index.astype(float)

    color = []
    marker = []
    accuracy = []
    
    for x in range(0,len(predict_result)):
        if true_data.values[x] == predict_result[x]:
            color.extend(['blue'])
            accuracy.extend([1])

        else:
            color.extend(['red'])
            accuracy.extend([0])


    plt.scatter(index.index.values, index["Rate"].values, color=color)

    plt.xticks(rotation=90)
    plt.show()
    
    return accuracy


def ensemble(model_names, moneyname, days):
    
#     df_now = get_now_data(datetime.datetime.now(), period_Days=days, money=moneyname, foot = 'D', back_days = NIGHT)###########
    
    
    
    df_now = get_now_data(datetime.datetime.now(), period_Days=days, money=moneyname, foot = FOOT, back_days = NIGHT)###########
    print("money: "+moneyname)
    count = [0,0]
    
    df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(days)+ "days ago")
    print(" LOW/MID/HIGH")
    print(count)

    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    values = []
    values_sum = []
    win=0
    lose=0
    answer = 0
    
    for x in range(len(model_names)):
        clf = pickle.load(open(model_names[x], 'rb')) #clfをload
        print(model_names[x])
        result = clf.predict(X_now) #predict
        
        precision = precision_score(y_now, result, average=None)
        #if(precision[0]!=0 and precision[1]!=0 and precision[0]!=1 and precision[1]!=1):
        if(1):
       
            #score表示
            print("accuracy_score: ", end="")
            print(accuracy_score(y_now, result))
            print("precision_score: ", end="")
            print(precision)
            print(confusion_matrix(y_now, result))

            profit, values = calc_profit(result, y_now) #予測結果について利率と、予測*利率による予測の信頼性を算出

            #answer += clf.predict(X_next)[0]*profit
            answer += clf.predict(X_next)[0]*accuracy_score(y_now, result)
            values = result*accuracy_score(y_now, result)
            if len(values_sum)==0:#1回目は配列をコピー
                values_sum = values
            else:
                values_sum += values #２回目からは加算
            print(values)
    
        #plot_result(X_now, result, y_now)
        
    print(values_sum)
    
    if(values_sum==[]):
        print("No model")
        return 0
    
    ensemble_result = np.where(values_sum<0, -1, 1) #クラス分類
    
    print("")
    print("ensemble_result: ",end='')
    print(accuracy_score(y_now.values, ensemble_result))
    print(ensemble_result)
    print(y_now.values)
    print(values_sum)
    print("precision_score: ", end="")
    print(precision_score(y_now, ensemble_result, average=None))
    print('confusion_matrix: ')
    print(confusion_matrix(y_now, ensemble_result))
    print("profit: ",end='')
    profit, values = calc_profit(ensemble_result, y_now)
    print(profit)
    print("next: ",end='')
    print(answer)
    if(answer>0):
        print("HIGH")
        hl = "HIGH"
    else:
        print("LOW")
        hl = "LOW"
    print(X_next.head(1))
    accuracy = plot_result(X_now, ensemble_result, y_now)
    
    global profit_sum
    profit_sum += profit
    
    global RESULT
    RESULT.extend([moneyname+": "+hl])
    return accuracy
        
    
def compere_ensemble(models_array, moneys_array, days):
    results = []
    
    for i in range(len(moneys_array)):
        results.append(np.array(ensemble(models_array[i], moneys_array[i], days)).astype('int'))
    
    np.savetxt('out.csv',results,delimiter=',')
    print(results)
    
def ensemble_set(models_array, moneys_array, days):
    
    for i in range(len(models_array)):
        print(ensemble(models_array[i], moneys_array[i], days))    

def plot_rates(days, moneys):
    
    result = []
    accord = []
    plt.figure(figsize=(40,20))
    for i in range(len(moneys)):
#         df_now = get_now_data(datetime.datetime.now(), period_Days=days, money=moneys[i], foot = 'D', back_days = 0)
        df_now = get_now_data(datetime.datetime.now(), period_Days=days, money=moneys[i], foot = FOOT, back_days = 0)
        plt.plot(df_now.index.values, df_now["Rate"].values)
        count = [0,0]
        df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
        result.append(df_now["class"].values)
    
    win = 0
    lose = 0
    
    for l in range(len(result[0])):
        temp = 0
        for m in range(len(moneys)):
            if result[m][l]==1:
                temp+=1
        if(temp==len(moneys)/2):
            accord.extend([-1])
            lose += len(moneys)/2
            win += len(moneys)/2
        else:
            accord.extend([temp])
            win += max(temp, len(moneys)-temp)
            lose += min(temp, len(moneys)-temp)
    
    
    plt.xticks(rotation=90)
    plt.hlines([0], 0, days, "blue", linestyles='dashed')     # hlines
    plt.show()
    print(accord)
    
    pay = 1000
    pay_sum = pay*len(moneys)
    
    print("win: ", win)
    print("lose: ", lose)
    print("rate: ", win/(win+lose))
    print("profit: ", (win*0.95-lose)*pay_sum)
    
def main():
    
    AUD_models = glob.glob('AUD_JPY*.sav')
    
    CAD_models = glob.glob('CAD_JPY*.sav')
    
    CHF_models = glob.glob('CHF_JPY*.sav')
    
    models = [AUD_models, CAD_models, CHF_models]
    moneys = ["AUD_JPY", "CAD_JPY", "CHF_JPY"]
    
    models = [glob.glob('USD_JPY*.sav'), glob.glob('EUR_JPY*.sav'), glob.glob('AUD_JPY*.sav'),
              glob.glob('GBP_JPY*.sav'), glob.glob('NZD_JPY*.sav'), glob.glob('CAD_JPY*.sav'),
              glob.glob('CHF_JPY*.sav')]#, glob.glob('ZAR_JPY*.sav')]
    moneys = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY']#, 'ZAR_JPY']
    
    #ensemble(CHF_models, "CHF_JPY", 60)   #選択した通貨に対してアンサンブル
    #compere_ensemble(models, moneys, 30)   #それぞれの通貨に対してアンサンブルした結果をcsvに返す
    #plot_rates(30,moneys)
    ensemble_set(models, moneys, DAYS*24)  #指定したmoneyで結果を出す
    
def get_csv_and_make_models():
    
    moneylist=['AUD_JPY', 'CAD_JPY', 'CHF_JPY']
    moneylist = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY']#, 'ZAR_JPY']
    #array100 = get_moneys_csv(back_days=5, days=100, moneylist = moneylist)
    array500 = get_moneys_csv(back_days=DAYS*2, hours=500, moneylist = moneylist) # 指定期間のcsvをsave
    array1000 = get_moneys_csv(back_days=DAYS*2, hours=1000, moneylist = moneylist)
    array1500 = get_moneys_csv(back_days=DAYS*2, hours=1500, moneylist = moneylist)
    array2000 = get_moneys_csv(back_days=DAYS*2, hours=2000, moneylist = moneylist)
   #make_models(test_days, namelist, moneylist, back_days, datanum):
    
    #make_models(24, array100, moneylist, 1, 100)
    make_models((DAYS-1)*24, array500, moneylist, DAYS, 500)
    make_models((DAYS-1)*24, array1000, moneylist, DAYS, 1000)
    make_models((DAYS-1)*24, array1500, moneylist, DAYS, 1500)
    make_models((DAYS-1)*24, array2000, moneylist, DAYS, 2000)

NIGHT = 0
FOOT = 'H1'

DAYS = 3

PAST = 0

get_csv_and_make_models()
RESULT = []

profit_sum = 0
main()
print(RESULT)

print('')
print("profit_sum: ", end='')
print(profit_sum)

print("pay 1000: ", end=str(profit_sum*1000))





"USD_JPY_2020-11-11_2020-12-02_500.csv", "EUR_JPY_2020-11-11_2020-12-02_500.csv", "AUD_JPY_2020-11-11_2020-12-02_500.csv", "GBP_JPY_2020-11-11_2020-12-02_500.csv", "NZD_JPY_2020-11-11_2020-12-02_500.csv", "CAD_JPY_2020-11-11_2020-12-02_500.csv", "CHF_JPY_2020-11-11_2020-12-02_500.csv", "USD_JPY_2020-10-21_2020-12-02_1000.csv", "EUR_JPY_2020-10-21_2020-12-02_1000.csv", "AUD_JPY_2020-10-21_2020-12-02_1000.csv", "GBP_JPY_2020-10-21_2020-12-02_1000.csv", "NZD_JPY_2020-10-21_2020-12-02_1000.csv", "CAD_JPY_2020-10-21_2020-12-02_1000.csv", "CHF_JPY_2020-10-21_2020-12-02_1000.csv", "USD_JPY_2020-09-30_2020-12-02_1500.csv", "EUR_JPY_2020-09-30_2020-12-02_1500.csv", "AUD_JPY_2020-09-30_2020-12-02_1500.csv", "GBP_JPY_2020-09-30_2020-12-02_1500.csv", "NZD_JPY_2020-09-30_2020-12-02_1500.csv", "CAD_JPY_2020-09-30_2020-12-02_1500.csv", "CHF_JPY_2020-09-30_2020-12-02_1500.csv", "USD_JPY_2020-09-09_2020-12-02_2000.csv", "EUR_JPY_2020-09-09_2020-12-02_2000.csv", "AUD_JPY_2020-09-09_2020-12-02_2000.csv",

C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


RBF SVM
Decision Tree
Random Forest
AdaBoost
Naive Bayes
Linear Discriminant Analysis
Quadratic Discriminant Analysis
[]
EUR_JPY_2020-11-11_2020-12-02_500.csv
Train Data
 LOW/HIGH
[0, 0]
money: EUR_JPY
NOW Data from48days ago
 LOW/MID/HIGH
[0, 0]
learning data
['2020-11-11 13:00'] to ['2020-12-02 00:00']

test data
['2020-12-03 09:00'] to ['2020-12-05 00:00']

Nearest Neighbors
Linear SVM


C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stana\.conda\envs\predict_rate\lib\site-

RBF SVM
Decision Tree
Random Forest
AdaBoost
Naive Bayes
Linear Discriminant Analysis
Quadratic Discriminant Analysis
[]
AUD_JPY_2020-11-11_2020-12-02_500.csv
Train Data
 LOW/HIGH
[0, 0]
money: AUD_JPY
NOW Data from48days ago
 LOW/MID/HIGH
[0, 0]
learning data
['2020-11-11 13:00'] to ['2020-12-02 00:00']

test data
['2020-12-03 09:00'] to ['2020-12-05 00:00']

Nearest Neighbors
Linear SVM


C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stana\.conda\envs\predict_rate\lib\site-

RBF SVM
Decision Tree
Random Forest
AdaBoost
Naive Bayes
Linear Discriminant Analysis
Quadratic Discriminant Analysis
[]
GBP_JPY_2020-11-11_2020-12-02_500.csv
Train Data
 LOW/HIGH
[0, 0]
money: GBP_JPY
NOW Data from48days ago
 LOW/MID/HIGH
[0, 0]
learning data
['2020-11-11 13:00'] to ['2020-12-02 00:00']

test data
['2020-12-03 09:00'] to ['2020-12-05 00:00']

Nearest Neighbors
Linear SVM


In [ ]:
62data
['USD_JPY: LOW', 'EUR_JPY: LOW', 'AUD_JPY: HIGH', 'GBP_JPY: LOW', 'NZD_JPY: HIGH', 'CHF_JPY: HIGH']

profit_sum: 25.049999999999997
pay 1000: 25049.999999999996
    

RSI 59data
['USD_JPY: LOW', 'EUR_JPY: LOW', 'AUD_JPY: HIGH', 'GBP_JPY: LOW', 'NZD_JPY: HIGH', 'CAD_JPY: HIGH', 'CHF_JPY: LOW']

profit_sum: 13.949999999999998
pay 1000: 13949.999999999998

RSI 59data
['USD_JPY: LOW', 'EUR_JPY: LOW', 'AUD_JPY: LOW', 'GBP_JPY: LOW', 'NZD_JPY: LOW', 'CAD_JPY: HIGH', 'CHF_JPY: LOW']

profit_sum: 8.099999999999998
pay 1000: 8099.999999999998
    
RSI 60data
[1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0]
['USD_JPY: LOW', 'EUR_JPY: HIGH', 'AUD_JPY: HIGH', 'GBP_JPY: HIGH', 'NZD_JPY: HIGH', 'CAD_JPY: HIGH', 'CHF_JPY: LOW']

profit_sum: 2.2499999999999956
pay 1000: 2249.9999999999955

RSI 56data
[1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0]
['USD_JPY: HIGH', 'EUR_JPY: LOW', 'AUD_JPY: HIGH', 'GBP_JPY: HIGH', 'NZD_JPY: LOW', 'CAD_JPY: LOW', 'CHF_JPY: LOW']

profit_sum: 8.099999999999996
pay 1000: 8099.999999999996

倍率調整　55data
['USD_JPY: LOW', 'EUR_JPY: HIGH', 'AUD_JPY: HIGH', 'GBP_JPY: HIGH', 'NZD_JPY: LOW', 'CHF_JPY: LOW']

profit_sum: 23.099999999999994
pay 1000: 23099.999999999993

倍率調整　63data    
[1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]
['USD_JPY: LOW', 'EUR_JPY: HIGH', 'AUD_JPY: HIGH', 'GBP_JPY: LOW', 'NZD_JPY: HIGH', 'CHF_JPY: HIGH']

profit_sum: -0.3000000000000034
pay 1000: -300.00000000000335

倍率調整　54data
[1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1]
['USD_JPY: HIGH', 'EUR_JPY: LOW', 'GBP_JPY: LOW', 'NZD_JPY: LOW', 'CAD_JPY: LOW', 'CHF_JPY: HIGH']

profit_sum: 5.549999999999997
pay 1000: 5549.999999999997

RSI remove 56data
[0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0]
['USD_JPY: LOW', 'EUR_JPY: HIGH', 'AUD_JPY: HIGH', 'GBP_JPY: HIGH', 'NZD_JPY: LOW', 'CAD_JPY: HIGH', 'CHF_JPY: HIGH']

profit_sum: 2.2499999999999982
pay 1000: 2249.999999999998
